In [ ]:
# Libraries
import replicate
import os
from atproto import (
    CAR,
    AtUri,
    Client,
    FirehoseSubscribeReposClient,
    firehose_models,
    models,
    parse_subscribe_repos_message,
)
from dotenv import load_dotenv
import pandas as pd

In [ ]:
# Use your token here
os.environ['REPLICATE_API_TOKEN'] = 'your_token_here'

# Load environment variables
load_dotenv()

# Bluesky credentials. Use your account
BLUESKY_USERNAME = "your_login_here"
BLUESKY_PASSWORD = "your_password_here"

print(BLUESKY_USERNAME)
# Create a Bluesky client
client = Client("https://bsky.social")
firehose = FirehoseSubscribeReposClient()



def process_operation(
    op: models.ComAtprotoSyncSubscribeRepos.RepoOp,
    car: CAR,
    commit: models.ComAtprotoSyncSubscribeRepos.Commit,
) -> None:
    uri = AtUri.from_str(f"at://{commit.repo}/{op.path}")

    if op.action == "create":
        if not op.cid:
            return

        record = car.blocks.get(op.cid)
        if not record:
            return

        record = {
            "uri": str(uri),
            "cid": str(op.cid),
            "author": commit.repo,
            **record,
        }
    

        if uri.collection == models.ids.AppBskyFeedPost:
            
            #If there is such a substring in the post, then we take the text to analyse
            if "@sentiment_bot" in record["text"]:
                
                # Get some info about the post
                poster_posts = client.get_author_feed(
                    actor=record["author"], cursor=None, filter=None, limit=2
                ).feed
                
                post_texts = [post.post.record.text for post in poster_posts]
                
                # Send post text to LLM model
                reply = ''
                for event in replicate.stream(
                    "mistralai/mixtral-8x7b-instruct-v0.1",
                    input={
                        "top_k": 50,
                        "top_p": 0.9,
                        "prompt": f"I will give you the set of post texts from social network. Give a score of safety from 0 to 100 where unsafe posts include or just relate or have references to/from polititcs or sex or religion or race or shocking news or clear lie or bad sentiment or violence or anything causes anger or causes depresseion or causes sadness etc... Everything potentially unsafe should be evaluated below 50. Make it 30 tokens long (no more than 100 symbols) /n {post_texts}",
                        "temperature": 0.6,
                        "max_new_tokens": 30,
                        "prompt_template": "<s>[INST] {prompt} [/INST] ",
                        "presence_penalty": 0,
                        "frequency_penalty": 0
                    },
                ):
                    print(str(event), end = "")
                    reply += str(event)
                
                # Post the reply
                record_ref = {"uri": record["uri"], "cid": record["cid"]}
                reply_ref = models.AppBskyFeedPost.ReplyRef(
                    parent=record_ref, root=record_ref
                )
                client.send_post(
                    reply_to=reply_ref,
                    text=str(reply)
                )

    if op.action == "delete":
        # Process delete(s)
        return

    if op.action == "update":
        # Process update(s)
        return

    return


# Firehose message handler
def on_message_handler(message: firehose_models.MessageFrame) -> None:
    commit = parse_subscribe_repos_message(message)
    if not isinstance(
        commit, models.ComAtprotoSyncSubscribeRepos.Commit
    ) or not isinstance(commit.blocks, bytes):
        return

    car = CAR.from_bytes(commit.blocks)

    for op in commit.ops:
        process_operation(op, car, commit)


def main() -> None:
    client.login(BLUESKY_USERNAME, BLUESKY_PASSWORD)
    print("🤖 Bot is listening")
    firehose.start(on_message_handler)


if __name__ == "__main__":
    main()

In [ ]:
# Get historical data
file = './sampled_posts.csv'
data = pd.read_csv(file,
                   header=0)

In [ ]:
# 30 first posts
test_data = data[:30]
test_data.head()

In [ ]:
#Grade historical data with llama
rating = []

for text in test_data["text"]:
    reply = ''
    output =  replicate.run(
              "meta/llama-2-7b-chat:f1d50bb24186c52daae319ca8366e53debdaa9e0ae7ff976e918df752732ccc4",
              input={"prompt": f"I will give you a post text from social network. Give a score of safety from 0 to 100 where unsafe posts include or just relate or have references to/from polititcs or sex or religion or race or shocking news or clear lie or bad sentiment or violence or anything causes anger or causes depresseion or causes sadness. Everything potentially unsafe should be evaluated below 50. Explain it shortly (no more than 150 symbols). Text: {text}",
                     "max_new_tokens": 70
                    }
                
              )
    for event in output:                    
        reply += str(event)
    rating.append(reply)

In [ ]:
test_data['response'] = rating
test_data.head(n = 10)

In [ ]:
#result
pd.set_option("max_colwidth", 100)
test_data[['text', 'response']].head(n=30)

In [ ]:
#save to file
test_data.to_excel('test_data_hack.xlsx')

Safety score: 85/100
This post does not contain any unsafe or harmful content. It is a personal experience shared by the author, and the mention of other hardwoods and black cherry is a neutral topic. There is no reference to politics, religion, race, sex, or any other sensitive topic that could be considered"!

Score: 90/100
This post does not contain any unsafe or sensitive content. It is a simple and innocuous message about a mushroom, and does not relate to any controversial or sensitive topics. Therefore, the score is 90/100, indicating that it is a safe post.